In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import math
from tqdm import tqdm
from multiprocessing import Process, Pool
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
def convert_temporal_data(dataframe):
  '''
  Convert temporal variables from seconds to hours.

  This conversion is necessary. Otherwise, large predictor values (e.g., p^6)
  will be irrelevant for OLS.
  '''

  for column in ['p', 'r']:
    dataframe[column] = dataframe[column] / 3600

  return dataframe

def read_score_distribution(csv_file):
  columns = ['p', 'q', 'r', 'score']
  distribution = pd.read_csv(csv_file, names=columns,skiprows=1)
  distribution = convert_temporal_data(distribution)

  return distribution

In [4]:
def add_quadratic_predictors(dataframe):
  dataframe[['p2', 'q2', 'r2']] = dataframe[['p', 'q', 'r']]**2
  dataframe['pq'] = dataframe['p']*dataframe['q']

  return dataframe

def add_cubic_predictors(dataframe):
  dataframe[['p3', 'q3', 'r3']] = dataframe[['p', 'q', 'r']]**3
  dataframe['p2q'] = dataframe['p']**2 * dataframe['q']
  dataframe['pq2'] = dataframe['p'] * dataframe['q']**2

  return dataframe

def add_quartic_predictiors(dataframe):
  dataframe[['p4', 'q4', 'r4']] = dataframe[['p', 'q', 'r']]**4
  dataframe['p3q'] = dataframe['p']**3 * dataframe['q']
  dataframe['p2q2'] = dataframe['p']**2 * dataframe['q']**2
  dataframe['pq3'] = dataframe['p'] * dataframe['q']**3

  return dataframe

def add_quintic_predictors(dataframe):
  dataframe[['p5', 'q5', 'r5']] = dataframe[['p', 'q', 'r']]**5
  dataframe['p4q'] = dataframe['p']**4 * dataframe['q']
  dataframe['p3q2'] = dataframe['p']**3 * dataframe['q']**2
  dataframe['p2q3'] = dataframe['p']**2 * dataframe['q']**3
  dataframe['pq4'] = dataframe['p'] * dataframe['q']**4

  return dataframe

def add_sextic_predictors(dataframe):
  dataframe[['p6', 'q6', 'r6']] = dataframe[['p', 'q', 'r']]**6
  dataframe['p5q'] = dataframe['p']**5 * dataframe['q']
  dataframe['p4q2'] = dataframe['p']**4 * dataframe['q']**2
  dataframe['p3q3'] = dataframe['p']**3 * dataframe['q']**3
  dataframe['p2q4'] = dataframe['p']**2 * dataframe['q']**4
  dataframe['pq5'] = dataframe['p'] * dataframe['q']**5

  return dataframe
def only_att_predictors(dataframe):
    dataframe[['p2', 'q2', 'r2']] = dataframe[['p', 'q', 'r']]**2
    dataframe[['p3', 'q3', 'r3']] = dataframe[['p', 'q', 'r']]**3
    dataframe[['p4', 'q4', 'r4']] = dataframe[['p', 'q', 'r']]**4
    dataframe[['p5', 'q5', 'r5']] = dataframe[['p', 'q', 'r']]**5
    dataframe[['p6', 'q6', 'r6']] = dataframe[['p', 'q', 'r']]**6

    return dataframe
def only_wq_att_predictors(dataframe):
    dataframe[['p2', 'r2']] = dataframe[['p', 'r']]**2
    dataframe[['p3',  'r3']] = dataframe[['p', 'r']]**3
    dataframe[['p4',  'r4']] = dataframe[['p', 'r']]**4
    dataframe[['p5',  'r5']] = dataframe[['p', 'r']]**5
    dataframe[['p6',  'r6']] = dataframe[['p', 'r']]**6

    return dataframe
def only_lin_predictors(dataframe):
    dataframe['pq'] = dataframe['p']*dataframe['q']
    dataframe['p2q'] = dataframe['p']**2 * dataframe['q']
    dataframe['pq2'] = dataframe['p'] * dataframe['q']**2
    dataframe['p3q'] = dataframe['p']**3 * dataframe['q']
    dataframe['p2q2'] = dataframe['p']**2 * dataframe['q']**2
    dataframe['pq3'] = dataframe['p'] * dataframe['q']**3
    dataframe['p4q'] = dataframe['p']**4 * dataframe['q']
    dataframe['p3q2'] = dataframe['p']**3 * dataframe['q']**2
    dataframe['p2q3'] = dataframe['p']**2 * dataframe['q']**3
    dataframe['pq4'] = dataframe['p'] * dataframe['q']**4

    return dataframe
def only_lin_predictors_pr(dataframe):
    dataframe['pr'] = dataframe['p']*dataframe['r']
    dataframe['p2r'] = dataframe['p']**2 * dataframe['r']
    dataframe['pr2'] = dataframe['p'] * dataframe['r']**2
    dataframe['p3r'] = dataframe['p']**3 * dataframe['r']
    dataframe['p2r2'] = dataframe['p']**2 * dataframe['r']**2
    dataframe['p4r'] = dataframe['p']**4 * dataframe['r']
    dataframe['p3r2'] = dataframe['p']**3 * dataframe['r']**2
    dataframe['p2r3'] = dataframe['p']**2 * dataframe['r']**3
    dataframe['pr4'] = dataframe['p'] * dataframe['r']**4

    return dataframe

In [5]:
def create_quadratic_polynomial(dataframe):
  dataframe = add_quadratic_predictors(dataframe)
  return dataframe

def create_cubic_polynomial(dataframe):
  dataframe = add_cubic_predictors(
              add_quadratic_predictors(dataframe))
  return dataframe

def create_quartic_polynomial(dataframe):
  dataframe = add_quartic_predictiors(
              add_cubic_predictors(
              add_quadratic_predictors(dataframe)))
  return dataframe

def create_quintic_polynomial(dataframe):
  dataframe = add_quintic_predictors(
              add_quartic_predictiors(
              add_cubic_predictors(
              add_quadratic_predictors(dataframe))))
  return dataframe

def create_sextic_polynomial(dataframe):
  dataframe = add_sextic_predictors(
              add_quintic_predictors(
              add_quartic_predictiors(
              add_cubic_predictors(
              add_quadratic_predictors(dataframe)))))
  return dataframe

In [33]:
variables = ['p', 'q', 'r']
degre_max = 4

# Générer les combinaisons
possibilites = []
possibilites.append("sqrt(p)")
possibilites.append("sqrt(q)")
possibilites.append("sqrt(r)")
for degre in range(1, degre_max + 1):
    for variable in variables:
        possibilites.append(variable + str(degre))

for degre in range(1, degre_max + 1):
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            possibilites.append(variables[i] + str(degre) + variables[j])
            if degre!=1:
                possibilites.append(variables[j] + str(degre) + variables[i])



# Afficher les combinaisons
print (possibilites)

['sqrt(p)', 'sqrt(q)', 'sqrt(r)', 'p1', 'q1', 'r1', 'p2', 'q2', 'r2', 'p3', 'q3', 'r3', 'p4', 'q4', 'r4', 'p1q', 'p1r', 'q1r', 'p2q', 'q2p', 'p2r', 'r2p', 'q2r', 'r2q', 'p3q', 'q3p', 'p3r', 'r3p', 'q3r', 'r3q', 'p4q', 'q4p', 'p4r', 'r4p', 'q4r', 'r4q']


In [23]:
import copy

history=[]
names=[]
def grid_search (dataframe, n,threshold):
    best_vif=[100000000000000]*20
    best_set=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    comb=list(itertools.combinations(possibilites,n))
    
    for var_set in tqdm(comb, desc="Progression :" ):
        df_copy = pd.DataFrame()
        
        # Appliquer les transformations pour chaque variable dans l'ensemble
        for var in var_set:
            
            if var.startswith('sqrt'):
                
                root_var = var[-2]  # Récupérer la variable ('p', 'q' ou 'r') à partir de la variable sqrt
                df_copy[var] = dataframe[root_var].apply(math.sqrt)
                
            elif var.startswith(('p', 'q', 'r'))and len(var)==2:
                power = int(var[1:]) if len(var) > 1 else 1  # Récupérer l'exposant du polynôme
                base_var = var[0]  # Récupérer la variable ('p', 'q' ou 'r') à partir de la variable polynomiale
                df_copy[var] = dataframe[base_var] ** power
            elif var in [ 'p1q', 'p1r', 'q1r']:
                var_1, var_2 = var[0], var[2]  # Récupérer les deux variables ('p', 'q' ou 'r') de la combinaison
                df_copy[var] = dataframe[var_1]  * dataframe[var_2]
            elif var in ['p2q', 'q2p', 'p2r', 'r2p', 'q2r', 'r2q']:
                var_1, var_2 = var[0], var[2]  # Récupérer les deux variables ('p', 'q' ou 'r') de la combinaison
                df_copy[var] = (dataframe[var_1] **2) * dataframe[var_2]
            elif var in ['p3q', 'q3p', 'p3r', 'r3p', 'q3r', 'r3q']:
                var_1, var_2 = var[0], var[2]  # Récupérer les trois variables ('p', 'q' ou 'r') de la combinaison
                df_copy[var] = (dataframe[var_1]**3) * dataframe[var_2] 
            elif var in ['p4q', 'q4p', 'p4r', 'r4p', 'q4r', 'r4q']:
                var_1, var_2 = var[0], var[2] # Récupérer les quatre variables ('p', 'q' ou 'r') de la combinaison
                df_copy[var] = (dataframe[var_1]**4) * dataframe[var_2]
        res=compute_vif(df_copy)
        max=np.max(res['VIF'])
        history.append(max)
        if n==1 : 
            names.append(var_set[-1])
        else : 
            res= var_set[0]
            for var in var_set:
                if var != var_set[0]:
                    res += "-"+var
            names.append(res)
        b=False
        idx=0
        for best in best_vif:
            if max<best and max>=threshold:
                b=True
                break
            idx+=1
        if b==True:
            tmp_vif=best_vif.copy()
            tmp_set=best_set.copy()
            for i in range (idx+1,5):
                
                tmp_vif[i]=best_vif[i-1]
                tmp_set[i]=best_set[i-1]
            tmp_vif[idx]=max
            tmp_set[idx]=var_set
            best_vif=tmp_vif
            best_set=tmp_set
        
        
            
    with open("grid.csv", "a+") as res:
        res.write(str(n))
        res.write(',')
        res.write(str(best_vif))
        res.write(',')
        res.write(str(best_set))
        res.write('\n')
    print(best_vif)
    print(best_set)


In [25]:
def compute_vif(features):
    vif = pd.DataFrame()
    vif['VIF'] = [variance_inflation_factor(features.values, i) for i in range(features.shape[1])]
    vif['feature'] = features.columns
    return vif

In [27]:
csv_file = "data/global_training_data_GA.csv"
raw_dist = read_score_distribution(csv_file)

In [26]:
features_label = ['p', 'q', 'r']
target_label = ["score"]
features = raw_dist[features_label]
target = raw_dist[target_label]
features

,p,q,r
0,0.009444,2,4.613333
1,0.002222,32,4.627500
2,0.049167,16,4.908333
3,0.014444,1,4.928889
4,2.546944,128,4.931667
...,...,...,...
81339,3.888611,32,19.506111
81340,0.280833,64,19.533889
81341,0.001111,1,83.753056
81342,0.018889,2,83.766111


In [11]:
df_tmp=pd.DataFrame()
df_tmp['pq']=features['p']*features['q']
df_tmp['qp']=features['q']*features['p']
print(df_tmp)
compute_vif(df_tmp)

               pq          qp
0        0.018889    0.018889
1        0.071111    0.071111
2        0.786667    0.786667
3        0.014444    0.014444
4      326.008889  326.008889
...           ...         ...
81339  124.435556  124.435556
81340   17.973333   17.973333
81341    0.001111    0.001111
81342    0.037778    0.037778
81343    0.179444    0.179444

[81344 rows x 2 columns]


/home/djoser/.local/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,VIF,feature
0,inf,pq
1,inf,qp


In [12]:
compute_vif(features[['p','q','r']])

,VIF,feature
0,1.272781,p
1,1.252365,q
2,1.151622,r


In [32]:
import seaborn as sns
history=[]
names=[]
print("######################1#######################")
grid_search(features,3,1 )




######################1#######################


Progression :: 100%|██████████| 5456/5456 [03:23<00:00, 26.77it/s]

[1.0003153515821601, 1.0003947578512513, 1.0004074078936562, 1.0004750801006355, 1.0004796715280073, 1.0005403866829419, 1.0006459700614874, 1.0007519098072788, 1.000994363901229, 1.0009956317557047, 1.0011571105714707, 1.0012172551091552, 1.0018789646954167, 1.001890197000479, 1.0020398145591378, 1.002484713180795, 1.0028314044150282, 1.0042560744563713, 1.0042584803146177, 1.0045677786472833]
[('r4', 'q4p', 'p4r'), ('r4', 'q3p', 'p4r'), ('p4', 'r4', 'q4p'), ('p4', 'q4', 'r4'), ('p4', 'q4p', 'r4p'), ('p4', 'r4', 'q3p'), ('p4', 'q3p', 'r4p'), ('r4', 'q2p', 'p4r'), ('p4', 'r3p', 'q4p'), ('r4', 'p3r', 'q4p'), ('q4p', 'p4r', 'r4p'), ('q3p', 'p4r', 'r4p'), ('q4', 'r4', 'p4q'), ('q4', 'p4q', 'r4p'), ('r4', 'q2p', 'p3r'), ('q4p', 'p4r', 'r4q'), ('p3r', 'q4p', 'r4q'), ('q4', 'p3r', 'r4q'), ('r3q', 'q4p', 'p4r'), ('q3p', 'r3q', 'p4r')]


In [34]:
history=[]
names=[]
print("######################2#######################")
grid_search(features,3,7 )


######################2#######################


Progression :: 100%|██████████| 7140/7140 [04:29<00:00, 26.48it/s]

[7.030116555458655, 7.048382716001007, 7.060764887979526, 7.0653566305080355, 7.07312967391587, 7.171080672199681, 7.194766749974029, 7.224269784525663, 7.228762217036042, 7.236905323688716, 7.321325735124912, 7.328005582869794, 7.696154955043733, 7.879268982727759, 7.924653506767348, 8.017977057387746, 8.095479439063087, 8.148971257751583, 8.262057799130202, 8.264768482036008]
[('q1r', 'q2r', 'q4p'), ('sqrt(p)', 'sqrt(r)', 'r1'), ('r3', 'q1r', 'q2r'), ('sqrt(p)', 'q1r', 'q2r'), ('q3', 'p1q', 'p2q'), ('q1r', 'p2r', 'p3r'), ('p2', 'p2r', 'p3r'), ('p2', 'p3', 'r3p'), ('q3', 'q1r', 'q2r'), ('r2', 'p2r', 'p3r'), ('q4', 'p2q', 'q3p'), ('p2r', 'p3r', 'p4q'), ('p2q', 'p3q', 'q4p'), ('r2', 'r3', 'q4r'), ('p2r', 'p3r', 'r4p'), ('r2', 'q1r', 'q2r'), ('r2', 'r3', 'r2q'), ('r2q', 'q4r', 'r4q'), ('r2p', 'r3p', 'q4p'), ('r2p', 'r3p', 'q4r')]


In [35]:
history=[]
names=[]
print("######################3#######################")
grid_search(features,3,8 )

######################3#######################


Progression :: 100%|██████████| 7140/7140 [04:48<00:00, 24.78it/s]

[8.017977057387746, 8.04749863301937, 8.057028222765389, 8.081109744303072, 8.095479439063087, 8.148971257751583, 8.262057799130202, 8.264768482036008, 8.294205647800824, 8.311540442552763, 8.383456355207883, 8.48112628886997, 8.64446113642909, 8.843509441334502, 8.97552277309257, 9.657088511764403, 9.934339133570777, 10.44689792450661, 11.119026752175918, 11.64895586513516]
[('r2', 'q1r', 'q2r'), ('r2', 'r3', 'r2p'), ('p2', 'p3', 'p4q'), ('sqrt(p)', 'p1', 'p1r'), ('r2', 'r3', 'r2q'), ('r2q', 'q4r', 'r4q'), ('r2p', 'r3p', 'q4p'), ('r2p', 'r3p', 'q4r'), ('r2p', 'r3p', 'q3r'), ('r2p', 'r3p', 'p4q'), ('r2p', 'q2r', 'r3p'), ('p2q', 'r2p', 'r3p'), ('r2p', 'r3p', 'r3q'), ('r2p', 'r3p', 'r4q'), ('r2q', 'q3r', 'r4q'), ('p3q', 'p4q', 'r4q'), ('p3q', 'p4q', 'q4r'), ('p3q', 'p3r', 'p4q'), ('p3q', 'p4q', 'p4r'), ('r2q', 'r3q', 'p4r')]


In [36]:
history=[]
names=[]
print("######################4#######################")
grid_search(features,3,9 )

######################4#######################


Progression :: 100%|██████████| 7140/7140 [05:05<00:00, 23.39it/s]

[9.011365745326126, 9.034334195197909, 9.042986031604123, 9.048934343709899, 9.05175399598587, 9.109820428299956, 9.201790286743291, 9.655915222178951, 9.657088511764403, 9.710755373862451, 9.900470686813891, 9.934339133570777, 10.028056775535644, 10.44689792450661, 11.119026752175918, 11.64895586513516, 11.694106621710567, 11.734416141177602, 11.79984475290541, 11.96886212553574]
[('sqrt(p)', 'p1', 'p4r'), ('p1q', 'q2p', 'q3r'), ('p2q', 'p3q', 'q3p'), ('sqrt(r)', 'p1q', 'q2p'), ('p1q', 'q2p', 'p2r'), ('r2p', 'r3p', 'p4r'), ('p1q', 'q2p', 'q4r'), ('r4', 'p3q', 'p4q'), ('p3q', 'p4q', 'r4q'), ('r2p', 'p3r', 'r3p'), ('p1q', 'p1r', 'q2p'), ('p3q', 'p4q', 'q4r'), ('q2r', 'p3q', 'p4q'), ('p3q', 'p3r', 'p4q'), ('p3q', 'p4q', 'p4r'), ('r2q', 'r3q', 'p4r'), ('q2', 'q4', 'p4q'), ('r2q', 'r3p', 'r3q'), ('p2r', 'r2q', 'r3q'), ('p2q', 'r2q', 'r3q')]


In [37]:
print("######################5#######################")
grid_search(features,3,10 )

######################5#######################


Progression :: 100%|██████████| 7140/7140 [04:56<00:00, 24.07it/s]

[10.028056775535644, 10.030228330855094, 10.03939593453847, 10.057838453114192, 10.123376926584458, 10.44689792450661, 10.91315023714512, 10.955586752513634, 11.119026752175918, 11.242493728592175, 11.64895586513516, 11.68932778339787, 11.690569913872517, 11.694106621710567, 11.734416141177602, 11.79984475290541, 11.96886212553574, 11.980907649157231, 12.177762309695117, 12.332842811380003]
[('q2r', 'p3q', 'p4q'), ('p2', 'p3q', 'p4q'), ('q1', 'p1q', 'p2q'), ('q1r', 'p3q', 'p4q'), ('sqrt(p)', 'sqrt(r)', 'p1'), ('p3q', 'p3r', 'p4q'), ('r1', 'r2', 'r4'), ('q2', 'p3q', 'p4q'), ('p3q', 'p4q', 'p4r'), ('q2r', 'r2q', 'r4q'), ('r2q', 'r3q', 'p4r'), ('q2', 'q4', 'r3p'), ('q2', 'q4', 'r3q'), ('q2', 'q4', 'p4q'), ('r2q', 'r3p', 'r3q'), ('p2r', 'r2q', 'r3q'), ('p2q', 'r2q', 'r3q'), ('r2q', 'r3q', 'r4p'), ('r2p', 'r2q', 'r3q'), ('r2q', 'r3q', 'q4p')]


In [12]:
history=[]
names=[]
grid_search(features,2 )
plt.figure(figsize=(40, 10))
data=pd.DataFrame(zip(history,names),columns=['VIF','NAME'])
data=data.sort_values('VIF')

Progression :: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 630/630 [00:11<00:00, 56.43it/s]

[1.0000000687381108, 1.0000003845348913, 1.0000007016175474, 1.0000009714029523, 1.000001195608339]
[('p4', 'r4q'), ('p4', 'r3q'), ('r4', 'p4q'), ('p4q', 'r4q'), ('p4', 'r4')]


<Figure size 4000x1000 with 0 Axes>

In [33]:
print(np.max(compute_vif(features)["VIF"]))

1.2727812286448872


In [119]:
compute_vif(features['p']*features['q'])


IndexError: tuple index out of range

In [21]:
p=[]
for i in range (2,3):
    p.append(Process(target=grid_search,args=(features,i)))
    p[i-2].start()
    
for proc in p:
    proc.join()
    
        

Progression: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 435/435 [00:35<00:00, 12.22it/s]


1.2862065211525044
('r3q', 'r4p')


In [ ]:
res=only_lin_predictors_pr(features)
vif=compute_vif(res)
print (vif)
print("mean = ",np.mean(vif.VIF))

In [ ]:
compute_vif(create_quadratic_polynomial(features))

In [ ]:
compute_vif(create_cubic_polynomial(features))

In [ ]:
compute_vif(create_quartic_polynomial(features))

In [ ]:
compute_vif(create_quintic_polynomial(features))

In [ ]:
compute_vif(features)